<a href="https://colab.research.google.com/github/Sombut-git/aiforthai-linebot-workshop-codespace/blob/main/Reuters_21578_TF_IDF_rtf_icf_icsdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"


from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pandas as pd
import re
import nltk


NumFeatures = 2000
MAX_DF = 0.5
MIN_DF = 2

nltk.download('wordnet')
nltk.download('stopwords')

# 1. โหลดข้อมูลข่าว Reuters-21578
import pandas as pd

# --- กรณีที่ไฟล์อยู่ในโฟลเดอร์ /content ของ Colab ---
df = pd.read_csv('/content/ModApte_train_test.csv')  # ระบุชื่อไฟล์ของคุณแทน mydata.csv

# แสดงข้อมูลบางส่วน
print("ดู 5 แถวแรกของข้อมูล:")
print(df.head())

# แยก X, y
X = df['text']
y = df['topics']  # หรือ df['labels'] แล้วแต่ชื่อคอลัมน์


print(df.head(10))
print(df)
rows,cols = df.shape



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


ดู 5 แถวแรกของข้อมูล:
                                                text text_type  \
0  Showers continued throughout the week in\nthe ...    "NORM"   
1  The U.S. Agriculture Department\nreported the ...    "NORM"   
2  Argentine grain board figures show\ncrop regis...    "NORM"   
3  Moody's Investors Service Inc said it\nlowered...    "NORM"   
4  Champion Products Inc said its\nboard of direc...    "NORM"   

                                              topics lewis_split  \
0                                           'cocoa'      "TRAIN"   
1   'grain' 'wheat' 'corn' 'barley' 'oat' 'sorghum'      "TRAIN"   
2   'veg-oil' 'linseed' 'lin-oil' 'soy-oil' 'sun-...     "TRAIN"   
3                                                        "TRAIN"   
4                                            'earn'      "TRAIN"   

       cgis_split  old_id new_id                           places people orgs  \
0  "TRAINING-SET"  "5544"    "1"  ['el-salvador' 'usa' 'uruguay']     []   []   
1  "TRAINI

In [2]:
setofwords = [set() for i in range(rows)]
for ii in range(rows):
  print(y[ii])
  words = y[ii].replace("'", "").split()
  print(words)
  print('--------------------------------')
  for k in words:
    setofwords[ii].add(k)

 'cocoa' 
['cocoa']
--------------------------------
 'grain' 'wheat' 'corn' 'barley' 'oat' 'sorghum' 
['grain', 'wheat', 'corn', 'barley', 'oat', 'sorghum']
--------------------------------
 'veg-oil' 'linseed' 'lin-oil' 'soy-oil' 'sun-oil' 'soybean' 'oilseed'
 'corn' 'sunseed' 'grain' 'sorghum' 'wheat' 
['veg-oil', 'linseed', 'lin-oil', 'soy-oil', 'sun-oil', 'soybean', 'oilseed', 'corn', 'sunseed', 'grain', 'sorghum', 'wheat']
--------------------------------
  
[]
--------------------------------
 'earn' 
['earn']
--------------------------------
 'acq' 
['acq']
--------------------------------
 'earn' 
['earn']
--------------------------------
 'earn' 'acq' 
['earn', 'acq']
--------------------------------
 'earn' 
['earn']
--------------------------------
 'earn' 
['earn']
--------------------------------
 'earn' 
['earn']
--------------------------------
 'wheat' 'grain' 
['wheat', 'grain']
--------------------------------
  
[]
--------------------------------
 'copper' 
['coppe

In [3]:
X_new = []
y_new = []

for ii in range(rows):
  if( len(setofwords[ii]) > 0) and (len(setofwords[ii]) <2):
    if isinstance(X[ii], str): # Add this check
      X_new.append(X[ii])
      y_new.append(y[ii])

print(len(X_new))
print(len(y_new))

rowsnew = len(X_new)

8342
8342


In [7]:
clslabeltset = []
[clslabeltset.append(xx) for xx in y_new if xx not in clslabeltset]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [8]:
for ii in range(len(clslabeltset)):
  print(clslabeltset[ii])

 'cocoa' 
 'earn' 
 'acq' 
 'copper' 
 'housing' 
 'coffee' 
 'sugar' 
 'trade' 
 'reserves' 
 'ship' 
 'money-supply' 
 'cotton' 
 'grain' 
 'crude' 
 'nat-gas' 
 'cpi' 
 'interest' 
 'money-fx' 
 'alum' 
 'tin' 
 'gold' 
 'strategic-metal' 
 'retail' 
 'ipi' 
 'oilseed' 
 'iron-steel' 
 'rubber' 
 'heat' 
 'jobs' 
 'lei' 
 'bop' 
 'gnp' 
 'zinc' 
 'veg-oil' 
 'orange' 
 'carcass' 
 'pet-chem' 
 'gas' 
 'wpi' 
 'livestock' 
 'lumber' 
 'instal-debt' 
 'meal-feed' 
 'lead' 
 'potato' 
 'nickel' 
 'inventories' 
 'cpu' 
 'l-cattle' 
 'silver' 
 'fuel' 
 'jet' 
 'income' 
 'rand' 
 'platinum' 
 'dlr' 
 'wool' 
 'tea' 
 'groundnut' 
 'rice' 
 'yen' 
 'hog' 
 'naphtha' 
 'propane' 
 'trade' 'trade' 
 'coconut' 


In [9]:
listofTopics = ['earn', 'acq', 'money-fx', 'grain', 'crude', 'trade', 'interest', 'ship']

In [10]:
numoftopic = [ 0 for i in range(len(clslabeltset))]
for ii in range(len(clslabeltset)):
  for jj in range(rowsnew):
    if(clslabeltset[ii] == y_new[jj]):
      numoftopic[ii] = numoftopic[ii]+1
  #print(X_new[ii])



In [11]:
for ii in range(len(clslabeltset)):
  print(clslabeltset[ii]+', '+str(numoftopic[ii]))



 'cocoa' , 53
 'earn' , 3736
 'acq' , 2058
 'copper' , 44
 'housing' , 14
 'coffee' , 110
 'sugar' , 114
 'trade' , 300
 'reserves' , 39
 'ship' , 143
 'money-supply' , 95
 'cotton' , 24
 'grain' , 45
 'crude' , 325
 'nat-gas' , 33
 'cpi' , 60
 'interest' , 198
 'money-fx' , 247
 'alum' , 46
 'tin' , 27
 'gold' , 90
 'strategic-metal' , 15
 'retail' , 16
 'ipi' , 37
 'oilseed' , 9
 'iron-steel' , 37
 'rubber' , 38
 'heat' , 10
 'jobs' , 43
 'lei' , 10
 'bop' , 23
 'gnp' , 63
 'zinc' , 12
 'veg-oil' , 30
 'orange' , 18
 'carcass' , 11
 'pet-chem' , 19
 'gas' , 18
 'wpi' , 20
 'livestock' , 18
 'lumber' , 11
 'instal-debt' , 4
 'meal-feed' , 7
 'lead' , 8
 'potato' , 5
 'nickel' , 4
 'inventories' , 2
 'cpu' , 2
 'l-cattle' , 1
 'silver' , 4
 'fuel' , 11
 'jet' , 3
 'income' , 5
 'rand' , 1
 'platinum' , 3
 'dlr' , 3
 'wool' , 1
 'tea' , 5
 'groundnut' , 2
 'rice' , 1
 'yen' , 6
 'hog' , 1
 'naphtha' , 1
 'propane' , 1
 'trade' 'trade' , 1
 'coconut' , 1


In [12]:
X_New1 = []
y_new1 = []
for jj in range(rowsnew):
  current_topic = y_new[jj].strip().strip("'")
  print(current_topic)
  if current_topic in listofTopics:
    print('true')
    X_New1.append(X_new[jj])
    y_new1.append(current_topic)

rowsnew1 = len(X_New1)

print(rowsnew1)

cocoa
earn
true
acq
true
earn
true
earn
true
earn
true
earn
true
copper
earn
true
earn
true
earn
true
housing
earn
true
earn
true
earn
true
earn
true
earn
true
coffee
acq
true
sugar
trade
true
reserves
ship
true
earn
true
earn
true
earn
true
money-supply
ship
true
earn
true
earn
true
earn
true
acq
true
earn
true
earn
true
coffee
money-supply
money-supply
earn
true
earn
true
earn
true
earn
true
earn
true
earn
true
earn
true
acq
true
earn
true
money-supply
money-supply
money-supply
money-supply
money-supply
cotton
sugar
earn
true
money-supply
acq
true
money-supply
earn
true
earn
true
grain
true
acq
true
earn
true
crude
true
acq
true
earn
true
money-supply
money-supply
acq
true
acq
true
grain
true
earn
true
earn
true
earn
true
earn
true
earn
true
crude
true
earn
true
earn
true
earn
true
earn
true
earn
true
acq
true
acq
true
earn
true
acq
true
earn
true
earn
true
earn
true
earn
true
earn
true
earn
true
earn
true
nat-gas
earn
true
acq
true
acq
true
earn
true
crude
true
crude
true
earn
true


In [13]:
#Text Preprocessing
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X_New1)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X_New1[sen]))

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)

    # Converting to Lowercase
    document = document.lower()

    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)

    documents.append(document)

In [14]:
#2. คำนวณหาค่า word count
vectorizer = CountVectorizer(max_features=NumFeatures, min_df=MIN_DF, max_df=MAX_DF, stop_words='english')
#vectorizer = TfidfVectorizer(max_features=NumFeatures, stop_words='english', max_df=MAX_DF, min_df=MIN_DF)
#X = vectorizer.fit_transform(newsgroups.data)
wordcount_matrix = vectorizer.fit_transform(documents)#newsgroups.data)
#y = newsgroups.target


# 2. แปลงข้อความเป็นเวกเตอร์ TF-IDF
vectorizer = TfidfVectorizer(max_features=NumFeatures, stop_words='english', max_df=MAX_DF, min_df=MIN_DF)
#X = vectorizer.fit_transform(wordcount_matrix)
#X = vectorizer.fit_transform(newsgroups.data)
# แปลง Count → TF-IDF
tfidf_transformer = TfidfTransformer()
X = tfidf_transformer.fit_transform(wordcount_matrix)
y = y_new1 #newsgroups.target

print("ขนาดของ TF-IDF matrix:", X.shape)
print(X[0])
print(X[1])
print(X[100])
#print(X[1][0])
#ขนาดของ TF-IDF matrix: (18846, 75560)
#<Compressed Sparse Row sparse matrix of dtype 'float64'
#with 69 stored elements and shape (1, 75560)>
#ขนาดของ TF-IDF matrix: (18846, 2000)
#<Compressed Sparse Row sparse matrix of dtype 'float64'
	#with 45 stored elements and shape (1, 2000)>



ขนาดของ TF-IDF matrix: (7052, 2000)
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 22 stored elements and shape (1, 2000)>
  Coords	Values
  (0, 103)	0.14455406739020013
  (0, 120)	0.17363838401338919
  (0, 127)	0.15196266149731436
  (0, 329)	0.18734891871178133
  (0, 340)	0.19861743789040437
  (0, 342)	0.26841770818538774
  (0, 370)	0.25844172975625535
  (0, 423)	0.3154277271830891
  (0, 473)	0.18146444299046885
  (0, 548)	0.15364084142294668
  (0, 551)	0.095728358775795
  (0, 709)	0.19443422711954395
  (0, 1027)	0.16650136088754858
  (0, 1211)	0.1704708240833074
  (0, 1465)	0.17385046397025583
  (0, 1534)	0.29442578672746855
  (0, 1537)	0.13793523185354928
  (0, 1699)	0.20153122817133096
  (0, 1700)	0.30471970696894046
  (0, 1756)	0.19215954479874983
  (0, 1786)	0.25373400243203237
  (0, 1947)	0.28632073450190204
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 62 stored elements and shape (1, 2000)>
  Coords	Values
  (0, 1)	0.09433768332506422
  (0, 38)	0

In [15]:
clslabel = []
[clslabel.append(xx) for xx in y_new1 if xx not in clslabel]

num_cls = len(clslabel)

In [16]:
print(clslabel)
print(num_cls)

['earn', 'acq', 'trade', 'ship', 'grain', 'crude', 'interest', 'money-fx']
8


In [17]:
wordcount_matrixext = wordcount_matrix.toarray()

In [18]:
rows, cols = wordcount_matrixext.shape #(len(wordcount_matrix), len(wordcount_matrix[0]))
print('rows = ',rows)
print('cols = ',cols)
rtf_sidf_matrix = [[0 for j in range(cols)] for i in range(rows)]


rows =  7052
cols =  2000


In [19]:
import math
def countnk(tk):
    counttk = 0
    for i in range(rows):
        if(wordcount_matrixext[i][tk] > 0):
            counttk = counttk+1

    return counttk

In [20]:
import math
import numpy as np

# แปลง documents ให้เป็นความยาวล่วงหน้า (ไม่ต้อง split ซ้ำใน loop)
doc_lengths = np.array([len(doc.split()) for doc in documents])



In [21]:
# หาค่า maxFreq ของแต่ละแถว
max_freqs = np.max(wordcount_matrixext, axis=1)
print(max_freqs)


[ 2  6  3 ...  6 11  3]


In [22]:
# สร้าง tf_matrix (TF = term frequency normalized by document length)
tf_matrix = np.divide(wordcount_matrixext, doc_lengths[:, None], where=doc_lengths[:, None] != 0)

In [23]:
# สร้าง rtf_sidf_matrix
import numpy as np
countnk_array = np.array([countnk(j) for j in range(cols)])  # คำนวณครั้งเดียว

In [24]:
rtf_sidf_matrix = np.zeros_like(wordcount_matrixext, dtype=float)
for i in range(rows):
    if max_freqs[i] > 0:
        rtf_sidf_matrix[i, :] = (wordcount_matrixext[i, :] / max_freqs[i]) * np.log2(rows / (1 + countnk_array))

In [25]:
X_rtf_sidf = rtf_sidf_matrix

In [26]:
def find_indexCls(clsval1,clslabelset):
    print('find ', clsval1)
    print(clslabelset)
    for iin in range(len(clslabelset)):
        if(clsval1 == clslabelset[iin]):
            print('return val ', iin)
            return iin
        else:
            print('not found class value')
            return -1

In [27]:
def find_indexCls1(clsval1):
    cls_indx = -1
    for iin in range(len(clslabel)):
        if(clsval1 == clslabel[iin]):
           cls_indx = iin
    # print('clsindx = ',cls_indx)
    return cls_indx

In [28]:

tik = [[set() for i in range(cols)] for j in range(num_cls)]
for i in range(cols):
    #for k in range(num_cls):
        #clslabel_k = clslabel[k]
        #sumtfijk = 0.0
        for j in range(rows):
            # f.write(str(tf_matrix[j][i])+',')
            if(tf_matrix[j][i]>= 0.0001 ): #and y[j]==clslabel_k): #if(y[j] == clslabel_k):
                # f.write('('+str(i)+','+str(j)+') = '+str(tf_matrix[j][i]))
                # f.write('\n')
                clsindex = find_indexCls1(y[j])
                tik[clsindex][i].add(j)

In [29]:
ICF_t_i = [0 for i in range(cols)]
X_ICF = [[0 for i in range(cols)] for j in range(rows)]
NCK = [0 for i in range(num_cls)]

In [30]:

#Calculate ICF Term Weighting
for i in range(cols):
    sumnumofcls = 0.0
    for k in range(num_cls):
        print(tik[k][i])
        if(len(tik[k][i]) > 0):
            sumnumofcls = sumnumofcls +1.0

    print(sumnumofcls)
    ICF_t_i[i] = 1.0+ math.log2(num_cls/sumnumofcls)
    #CS_t_i[i] = 1.0+ math.log2(num_cls/sumnumofcls)

{6017, 770, 6660, 6, 1671, 4102, 399, 3986, 1683, 3605, 6677, 5657, 26, 6810, 1696, 5152, 930, 4772, 937, 5293, 4783, 6831, 6577, 5555, 6580, 5430, 5943, 5817, 1722, 5693, 6081, 6979, 714, 5194, 6617, 6493, 357, 1386, 363, 1524, 122, 6523}
{129, 774, 6288, 273, 6040, 3481, 4256, 5923, 6183, 1705, 4909, 3250, 2361, 6207, 4418, 2117, 4426, 2904, 3289, 230, 6378, 4075, 5483, 6251, 366, 238, 1137, 7026}
{1681}
{4578}
{5409, 5298}
{902, 6159, 656, 4634, 6938, 672, 6305, 6691, 6696, 43, 692, 6839, 439, 6584, 6982, 4092, 6141, 6782, 6143}
{5313, 4515, 4517, 4774, 4775, 6311, 5289, 4491, 5515, 6189, 4526, 5296, 5297, 3666, 4790, 4791, 4795}
{7042, 5029, 5484, 3890, 5492, 2134}
8.0
{2, 3, 7, 10, 11, 12, 13, 22, 23, 26, 33, 35, 52, 55, 65, 68, 70, 74, 80, 81, 83, 126, 139, 140, 143, 150, 151, 154, 161, 163, 164, 181, 183, 187, 190, 204, 207, 210, 212, 220, 221, 231, 236, 237, 243, 255, 256, 259, 267, 272, 275, 283, 291, 310, 316, 320, 321, 323, 324, 326, 327, 331, 335, 343, 344, 352, 353, 354, 3

In [31]:
X_ICF1 = [[0 for i in range(cols)] for j in range(rows)]
X_ICF2 = [[0 for i in range(cols)] for j in range(rows)]

In [32]:
 for kk in range(num_cls):
  sumx = 0.0
  for ii in range(cols):
    if(len(tik[k][i]) > 0):
      sumx = sumx +len(tik[kk][ii])
  NCK[kk] = sumx
  print(NCK[kk])

100554.0
80388.0
20822.0
6067.0
2297.0
19236.0
9141.0
13382.0


In [33]:
#Claculate find ICSDF
CS_t_i = [0 for i in range(cols)]
Cdel_ti = [0 for i in range(cols)]

In [34]:
for ii in range(cols):
  sumti = 0
  for kk in range(num_cls):
    sumti = sumti + (len(tik[kk][ii])/NCK[kk])
  Cdel_ti [ii] = sumti
  print(Cdel_ti [ii])

0.00514539743417324
0.035252341945800306
0.001040333497308027
0.0017742814523458745
0.00151441408526811
0.0016895453824190827
0.0016828975697038525
0.0018309426788095755
0.0012784945325098579
0.0013562224243392838
0.0013863959020859365
0.03125451354124087
0.010998125172552796
0.0006711021543445018
0.0009561077082056355
0.0011741206713717176
0.0010251623748077517
0.0010059896606984579
0.000769285621851516
0.0004905473628183665
0.0010921119957596878
0.0006670323762491673
0.014051528356281163
0.0010848718854957389
0.0009152348146306217
0.0013096034454792742
0.0004849972664909981
0.0006686857565110271
0.00035309535197397817
0.0009228083416729466
0.0004949421717160513
0.0006771315461665868
0.01628946251333764
0.002871007973009537
0.0003431163024258654
0.0007356949312257337
0.0009296420735428125
0.0006290335830703604
0.0017797064745375343
0.000407775258550241
0.0007916405247053179
0.0007508310078228936
0.01483683083848999
0.0010972011536341828
0.00048203564677162515
0.0007131893364392366
0.0

In [35]:
for ii in range(cols):
  CS_t_i[ii] = 1.0+ math.log2(num_cls/Cdel_ti[ii])
  print( CS_t_i[ii])

11.602501768443476
8.82613708541622
13.908738200732214
13.138549403717413
13.367024550222073
13.209149181800727
13.214836915586272
13.09319765916946
13.611338293764012
13.526190480689053
13.494444989368327
8.99979164181249
10.506598578278306
14.541179991143613
14.030539228559702
13.734203594235407
13.929931849295272
13.957168807088939
14.344193035298504
14.993319942790144
13.83866347287131
14.54995559127224
10.153129131776273
13.848259602428277
14.093570447691402
13.576654262098009
15.009735763447612
14.546383992348407
15.467654549469287
14.081681334310739
14.980452406785727
14.528276246537011
9.939917188252315
12.444226947636551
15.50901470550406
14.408604728005114
14.071037016144798
14.63457533726888
13.134144966306502
15.259938135950001
14.302866912572133
14.379224147722997
10.074673225974067
13.83195624046439
15.018572541145957
14.453427247508083
14.023052302280464
14.64359120642318
14.27726354158422
15.10207058827863
14.397086466193365
14.947081680826937
10.658405482288918
13.0971

In [36]:
XArray = X.toarray()

In [37]:
X_ICSDF = [[0 for i in range(cols)] for j in range(rows)]

In [38]:
for i in range(rows):
    for j in range(cols):
        X_ICF[i][j] = (1+XArray[i][j])*ICF_t_i[j]
        X_ICF1[i][j] = (tf_matrix[i][j]+XArray[i][j])*ICF_t_i[j]
        X_ICF2[i][j] = XArray[i][j]*ICF_t_i[j]
        X_ICSDF[i][j] =  (tf_matrix[i][j]+XArray[i][j])*CS_t_i[j]

In [39]:
# 3. แบ่งข้อมูลเป็น train/test
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
import numpy as np
import math

cv = KFold(n_splits=5, random_state=42, shuffle=True)
scoring = {'acc': 'accuracy',
           'prec_macro': 'precision_macro',
           'rec_macro': 'recall_macro',
           'f1_macro': 'f1_macro',
           'f1_micro': 'f1_micro'}


In [40]:


knnsize = 1
classifier1 =  KNeighborsClassifier(n_neighbors=knnsize)
scores1 = cross_validate(classifier1, X, y, scoring=scoring, cv=cv, n_jobs=1,return_train_score=True)
print('**********result tf-idf**********')
print('\n')
print ('k_NN, k = '+str(knnsize )+'\n')
#f.write(scores.keys())
print('acc = '+str("%.3f" % np.mean(scores1['test_acc']))+'+/-'+str("%.3f" % np.std(scores1['test_acc'])))
print('\n')
print('recall = '+str("%.3f" % np.mean(scores1['test_rec_macro'])))
print('\n')
print('precision = '+str("%.3f" % np.mean(scores1['test_prec_macro'])))
print('\n')
print('f1 = '+str("%.3f" % np.mean(scores1['test_f1_macro'])))
print('\n')

**********result tf-idf**********


k_NN, k = 1

acc = 0.926+/-0.007


recall = 0.847


precision = 0.874


f1 = 0.856




In [225]:
knnsize = 1
classifier11 =  KNeighborsClassifier(n_neighbors=knnsize)
scores11 = cross_validate(classifier11, X_rtf_sidf, y, scoring=scoring, cv=cv, n_jobs=-1,
                         return_train_score=True)
print('**********result tf-rtf-sisf**********')
print('\n')
print ('k_NN, k = '+str(knnsize )+'\n')
#f.write(scores.keys())
print('acc = '+str("%.3f" % np.mean(scores11['test_acc']))+'+/-'+str("%.3f" % np.std(scores11['test_acc'])))
print('\n')
print('recall = '+str("%.3f" % np.mean(scores11['test_rec_macro'])))
print('\n')
print('precision = '+str("%.3f" % np.mean(scores11['test_prec_macro'])))
print('\n')
print('f1 = '+str("%.3f" % np.mean(scores11['test_f1_macro'])))
print('\n')

**********result tf-rtf-sisf**********


k_NN, k = 1

acc = 0.831+/-0.009


recall = 0.711


precision = 0.859


f1 = 0.767




In [226]:
knnsize = 1
classifier12 =  KNeighborsClassifier(n_neighbors=knnsize)
scores12 = cross_validate(classifier12,  X_ICF, y, scoring=scoring, cv=cv, n_jobs=1,return_train_score=True)
print('**********result tf-idf-icf**********')
print('\n')
print ('k_NN, k = '+str(knnsize )+'\n')
#f.write(scores.keys())
print('acc = '+str("%.3f" % np.mean(scores12['test_acc']))+'+/-'+str("%.3f" % np.std(scores12['test_acc'])))
print('\n')
print('recall = '+str("%.3f" % np.mean(scores12['test_rec_macro'])))
print('\n')
print('precision = '+str("%.3f" % np.mean(scores12['test_prec_macro'])))
print('\n')
print('f1 = '+str("%.3f" % np.mean(scores12['test_f1_macro'])))
print('\n')

**********result tf-idf-icf**********


k_NN, k = 1

acc = 0.907+/-0.006


recall = 0.841


precision = 0.837


f1 = 0.832




In [227]:
knnsize = 1
classifier13 =  KNeighborsClassifier(n_neighbors=knnsize)
scores13 = cross_validate(classifier13,  X_ICF1, y, scoring=scoring, cv=cv, n_jobs=1,return_train_score=True)
print('**********result tf-idf-icf1**********')
print('\n')
print ('k_NN, k = '+str(knnsize )+'\n')
#f.write(scores.keys())
print('acc = '+str("%.3f" % np.mean(scores13['test_acc']))+'+/-'+str("%.3f" % np.std(scores13['test_acc'])))
print('\n')
print('recall = '+str("%.3f" % np.mean(scores13['test_rec_macro'])))
print('\n')
print('precision = '+str("%.3f" % np.mean(scores13['test_prec_macro'])))
print('\n')
print('f1 = '+str("%.3f" % np.mean(scores13['test_f1_macro'])))
print('\n')

**********result tf-idf-icf1**********


k_NN, k = 1

acc = 0.909+/-0.006


recall = 0.844


precision = 0.831


f1 = 0.832




In [229]:
knnsize = 1
classifier14 =  KNeighborsClassifier(n_neighbors=knnsize)
scores14 = cross_validate(classifier14,  X_ICF2, y, scoring=scoring, cv=cv, n_jobs=1,return_train_score=True)
print('**********result tf-idf-icf2**********')
print('\n')
print ('k_NN, k = '+str(knnsize )+'\n')
#f.write(scores.keys())
print('acc = '+str("%.3f" % np.mean(scores14['test_acc']))+'+/-'+str("%.3f" % np.std(scores14['test_acc'])))
print('\n')
print('recall = '+str("%.3f" % np.mean(scores14['test_rec_macro'])))
print('\n')
print('precision = '+str("%.3f" % np.mean(scores14['test_prec_macro'])))
print('\n')
print('f1 = '+str("%.3f" % np.mean(scores14['test_f1_macro'])))
print('\n')

**********result tf-idf-icf2**********


k_NN, k = 1

acc = 0.907+/-0.006


recall = 0.841


precision = 0.837


f1 = 0.832




In [230]:
knnsize = 1
classifier15 =  KNeighborsClassifier(n_neighbors=knnsize)
scores15 = cross_validate(classifier15,  X_ICSDF, y, scoring=scoring, cv=cv, n_jobs=1,return_train_score=True)
print('**********result tf-idf-icsdf**********')
print('\n')
print ('k_NN, k = '+str(knnsize )+'\n')
#f.write(scores.keys())
print('acc = '+str("%.3f" % np.mean(scores15['test_acc']))+'+/-'+str("%.3f" % np.std(scores15['test_acc'])))
print('\n')
print('recall = '+str("%.3f" % np.mean(scores15['test_rec_macro'])))
print('\n')
print('precision = '+str("%.3f" % np.mean(scores15['test_prec_macro'])))
print('\n')
print('f1 = '+str("%.3f" % np.mean(scores15['test_f1_macro'])))
print('\n')

**********result tf-idf-icsdf**********


k_NN, k = 1

acc = 0.916+/-0.004


recall = 0.842


precision = 0.847


f1 = 0.840




In [42]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

In [232]:
classifier20 =  GaussianNB()
#classifier11 =KNeighborsClassifier(n_neighbors=knnsize)
#classifier11 =  RandomForestClassifier()

scores20 = cross_validate(classifier20,X.toarray(), y, scoring=scoring, cv=cv, n_jobs=-1,
                          return_train_score=True)
# report performance
print('**********Navie Bayes**********')
print('**********result tf-idf**********')
print('\n')
#print(scores.keys())
print('acc = '+str("%.3f" % np.mean(scores20['test_acc'])))
print('+/-'+str("%.3f" % np.std(scores20['test_acc'])))
print('\n')
print('recall = '+str("%.3f" % np.mean(scores20['test_rec_macro'])))
print('\n')
print('precision = '+str("%.3f" % np.mean(scores20['test_prec_macro'])))
print('\n')
print('f1 = '+str("%.3f" % np.mean(scores20['test_f1_macro'])))
print('\n')

**********Navie Bayes**********
**********result tf-idf**********


acc = 0.851
+/-0.008


recall = 0.588


precision = 0.765


f1 = 0.627




In [233]:
classifier21 =  GaussianNB()
#classifier11 =KNeighborsClassifier(n_neighbors=knnsize)
#classifier11 =  RandomForestClassifier()

scores21 = cross_validate(classifier21,X_rtf_sidf, y, scoring=scoring, cv=cv, n_jobs=-1,
                          return_train_score=True)
# report performance
print('********** Navie Bayes **********')
print('********** result rtf-sisf **********')
print('\n')
#print(scores.keys())
print('acc = '+str("%.3f" % np.mean(scores21['test_acc']))+'+/-'+str("%.3f" % np.std(scores21['test_acc'])))
print('\n')
print('recall = '+str("%.3f" % np.mean(scores21['test_rec_macro'])))
print('\n')
print('precision = '+str("%.3f" % np.mean(scores21['test_prec_macro'])))
print('\n')
print('f1 = '+str("%.3f" % np.mean(scores21['test_f1_macro'])))
print('\n')

********** Navie Bayes **********
********** result rtf-sisf **********


acc = 0.832+/-0.009


recall = 0.578


precision = 0.739


f1 = 0.611




In [234]:

classifier22 =  GaussianNB()
#classifier11 =KNeighborsClassifier(n_neighbors=knnsize)
#classifier11 =  RandomForestClassifier()

scores22 = cross_validate(classifier22,X_ICF, y, scoring=scoring, cv=cv, n_jobs=-1,
                          return_train_score=True)
# report performance
print('********** Navie Bayes **********')
print('********** result tf-idf-icf**********')
print('\n')
#print(scores.keys())
print('acc = '+str("%.3f" % np.mean(scores22['test_acc']))+' +/- '+str("%.3f" % np.std(scores22['test_acc'])))
print('\n')
print('recall = '+str("%.3f" % np.mean(scores22['test_rec_macro'])))
print('\n')
print('precision = '+str("%.3f" % np.mean(scores22['test_prec_macro'])))
print('\n')
print('f1 = '+str("%.3f" % np.mean(scores22['test_f1_macro'])))
print('\n')

********** Navie Bayes **********
********** result tf-idf-icf**********


acc = 0.859 +/- 0.008


recall = 0.628


precision = 0.791


f1 = 0.672




In [235]:
classifier23 =  GaussianNB()
#classifier11 =KNeighborsClassifier(n_neighbors=knnsize)
#classifier11 =  RandomForestClassifier()

scores23 = cross_validate(classifier23,X_ICF1, y, scoring=scoring, cv=cv, n_jobs=-1,
                          return_train_score=True)
# report performance
print('********** Navie Bayes **********')
print('********** result tf-idf-icf1**********')
print('\n')
#print(scores.keys())
print('acc = '+str("%.3f" % np.mean(scores23['test_acc']))+' +/- '+str("%.3f" % np.std(scores23['test_acc'])))
print('\n')
print('recall = '+str("%.3f" % np.mean(scores23['test_rec_macro'])))
print('\n')
print('precision = '+str("%.3f" % np.mean(scores23['test_prec_macro'])))
print('\n')
print('f1 = '+str("%.3f" % np.mean(scores23['test_f1_macro'])))
print('\n')

********** Navie Bayes **********
********** result tf-idf-icf1**********


acc = 0.859 +/- 0.007


recall = 0.628


precision = 0.793


f1 = 0.673




In [43]:
classifier24 =  GaussianNB()
#classifier11 =KNeighborsClassifier(n_neighbors=knnsize)
#classifier11 =  RandomForestClassifier()

scores24 = cross_validate(classifier24,X_ICF2, y, scoring=scoring, cv=cv, n_jobs=-1,
                          return_train_score=True)
# report performance
print('********** Navie Bayes **********')
print('********** result tf-idf-icf2**********')
print('\n')
#print(scores.keys())
print('acc = '+str("%.3f" % np.mean(scores24['test_acc']))+' +/- '+str("%.3f" % np.std(scores24['test_acc'])))
print('\n')
print('recall = '+str("%.3f" % np.mean(scores24['test_rec_macro'])))
print('\n')
print('precision = '+str("%.3f" % np.mean(scores24['test_prec_macro'])))
print('\n')
print('f1 = '+str("%.3f" % np.mean(scores24['test_f1_macro'])))
print('\n')

********** Navie Bayes **********
********** result tf-idf-icf2**********


acc = 0.859 +/- 0.008


recall = 0.628


precision = 0.791


f1 = 0.672




In [47]:

classifier25 =  GaussianNB()
#classifier11 =KNeighborsClassifier(n_neighbors=knnsize)
#classifier11 =  RandomForestClassifier()

scores25 = cross_validate(classifier25,X_ICSDF, y, scoring=scoring, cv=cv, n_jobs=-1,
                          return_train_score=True)
# report performance
print('********** Navie Bayes **********')
print('********** result tf-idf-icsdf**********')
print('\n')
#print(scores.keys())
print('acc = '+str("%.3f" % np.mean(scores25['test_acc']))+' +/- '+str("%.3f" % np.std(scores25['test_acc'])))
print('\n')
print('recall = '+str("%.3f" % np.mean(scores25['test_rec_macro'])))
print('\n')
print('precision = '+str("%.3f" % np.mean(scores25['test_prec_macro'])))
print('\n')
print('f1 = '+str("%.3f" % np.mean(scores25['test_f1_macro'])))
print('\n')

********** Navie Bayes **********
********** result tf-idf-icsdf**********


acc = 0.849 +/- 0.008


recall = 0.581


precision = 0.764


f1 = 0.620




In [48]:

classifier30 =  RandomForestClassifier()

scores30 = cross_validate(classifier30,X.toarray(), y, scoring=scoring, cv=cv, n_jobs=-1,
                          return_train_score=True)
# report performance
print('********** Random forest **********')
print('**********result tf-idf**********')
print('\n')
#print(scores.keys())
print('acc = '+str("%.3f" % np.mean(scores30['test_acc']))+'+/-'+str("%.3f" % np.std(scores30['test_acc'])))
print('\n')
print('recall = '+str("%.3f" % np.mean(scores30['test_rec_macro'])))
print('\n')
print('precision = '+str("%.3f" % np.mean(scores30['test_prec_macro'])))
print('\n')
print('f1 = '+str("%.3f" % np.mean(scores30['test_f1_macro'])))
print('\n')

********** Random forest **********
**********result tf-idf**********


acc = 0.949+/-0.005


recall = 0.804


precision = 0.941


f1 = 0.844




In [49]:
classifier31 =  RandomForestClassifier()

scores31 = cross_validate(classifier31,X_rtf_sidf, y, scoring=scoring, cv=cv, n_jobs=-1,
                          return_train_score=True)
# report performance
print('********** Random forest **********')
print('**********result rtf-sisf**********')
print('\n')
#print(scores.keys())
print('acc = '+str("%.3f" % np.mean(scores31['test_acc']))+' +/- '+str("%.3f" % np.std(scores31['test_acc'])))
print('\n')
print('recall = '+str("%.3f" % np.mean(scores31['test_rec_macro'])))
print('\n')
print('precision = '+str("%.3f" % np.mean(scores31['test_prec_macro'])))
print('\n')
print('f1 = '+str("%.3f" % np.mean(scores31['test_f1_macro'])))
print('\n')

********** Random forest **********
**********result rtf-sisf**********


acc = 0.947 +/- 0.007


recall = 0.780


precision = 0.937


f1 = 0.822




In [50]:
classifier32 =  RandomForestClassifier()

scores32 = cross_validate(classifier32,X_ICF, y, scoring=scoring, cv=cv, n_jobs=-1,
                          return_train_score=True)
# report performance
print('********** Random forest **********')
print('**********result tf-idf-icf**********')
print('\n')
#print(scores.keys())
print('acc = '+str("%.3f" % np.mean(scores32['test_acc']))+' +/- '+str("%.3f" % np.std(scores32['test_acc'])))
print('\n')
print('recall = '+str("%.3f" % np.mean(scores32['test_rec_macro'])))
print('\n')
print('precision = '+str("%.3f" % np.mean(scores32['test_prec_macro'])))
print('\n')
print('f1 = '+str("%.3f" % np.mean(scores32['test_f1_macro'])))
print('\n')

********** Random forest **********
**********result tf-idf-icf**********


acc = 0.949 +/- 0.004


recall = 0.799


precision = 0.941


f1 = 0.841




In [44]:
classifier33 =  RandomForestClassifier()

scores33 = cross_validate(classifier33,X_ICF1, y, scoring=scoring, cv=cv, n_jobs=-1,
                          return_train_score=True)
# report performance
print('********** Random forest **********')
print('**********result tf-idf-icf1**********')
print('\n')
#print(scores.keys())
print('acc = '+str("%.3f" % np.mean(scores33['test_acc']))+' +/- '+str("%.3f" % np.std(scores33['test_acc'])))
print('\n')
print('recall = '+str("%.3f" % np.mean(scores33['test_rec_macro'])))
print('\n')
print('precision = '+str("%.3f" % np.mean(scores33['test_prec_macro'])))
print('\n')
print('f1 = '+str("%.3f" % np.mean(scores33['test_f1_macro'])))
print('\n')

********** Random forest **********
**********result tf-idf-icf1**********


acc = 0.948 +/- 0.006


recall = 0.796


precision = 0.941


f1 = 0.837




In [45]:
classifier34 =  RandomForestClassifier()

scores34 = cross_validate(classifier34,X_ICF2, y, scoring=scoring, cv=cv, n_jobs=-1,
                          return_train_score=True)
# report performance
print('********** Random forest **********')
print('**********result tf-idf-icf2**********')
print('\n')
#print(scores.keys())
print('acc = '+str("%.3f" % np.mean(scores34['test_acc']))+' +/- '+str("%.3f" % np.std(scores34['test_acc'])))
print('\n')
print('recall = '+str("%.3f" % np.mean(scores34['test_rec_macro'])))
print('\n')
print('precision = '+str("%.3f" % np.mean(scores34['test_prec_macro'])))
print('\n')
print('f1 = '+str("%.3f" % np.mean(scores34['test_f1_macro'])))
print('\n')

********** Random forest **********
**********result tf-idf-icf2**********


acc = 0.948 +/- 0.005


recall = 0.801


precision = 0.943


f1 = 0.845




In [46]:

classifier35 =  RandomForestClassifier()

scores35 = cross_validate(classifier35,X_ICSDF, y, scoring=scoring, cv=cv, n_jobs=-1,
                          return_train_score=True)
# report performance
print('********** Random forest **********')
print('**********result tf-idf-icsdf**********')
print('\n')
#print(scores.keys())
print('acc = '+str("%.3f" % np.mean(scores35['test_acc']))+' +/- '+str("%.3f" % np.std(scores35['test_acc'])))
print('\n')
print('recall = '+str("%.3f" % np.mean(scores35['test_rec_macro'])))
print('\n')
print('precision = '+str("%.3f" % np.mean(scores35['test_prec_macro'])))
print('\n')
print('f1 = '+str("%.3f" % np.mean(scores35['test_f1_macro'])))
print('\n')

********** Random forest **********
**********result tf-idf-icsdf**********


acc = 0.947 +/- 0.006


recall = 0.793


precision = 0.940


f1 = 0.835




In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# 4. สร้างโมเดล k-NN
# ใช้ metric = 'cosine' เพราะข้อความเหมาะกับ cosine similarity
knn = KNeighborsClassifier(n_neighbors=5, metric='cosine')
knn.fit(X_train, y_train)

# 5. ทำนายผล
y_pred = knn.predict(X_test)

In [ ]:
# 6. ประเมินประสิทธิภาพ
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=newsgroups.target_names))



In [ ]:
# 7. ทดสอบกับข่าวใหม่
sample_docs = [
    "The graphics card and processor are important for computer performance.",
    "Religion and spirituality have a big impact on society.",
    "The new government law changes healthcare policy."
]

X_new = vectorizer.transform(sample_docs)
predicted = knn.predict(X_new)

for doc, cat in zip(sample_docs, predicted):
    print(f"\nข่าว: {doc}\n → จัดเป็นหมวด: {newsgroups.target_names[cat]}")